In [ ]:
# -*- coding: utf-8 -*-
# ======================= 기본 임포트 =======================
import os, pickle, math, heapq, time
import osmnx as ox
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import contextily as cx
import pandas as pd
from geopy.distance import geodesic
from contextlib import contextmanager
from collections import OrderedDict
import atexit

# ===== GUI 백엔드 (Windows) =====
matplotlib.use('TkAgg')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

- 밑에껀 안쓰는거

In [ ]:

# ======================= 타이머 유틸 =======================
TIMINGS = OrderedDict()

@contextmanager
def timer(name):
    t0 = time.perf_counter()
    try:
        yield
    finally:
        dt = time.perf_counter() - t0
        TIMINGS[name] = TIMINGS.get(name, 0.0) + dt
        print(f"[⏱] {name}: {dt*1000:.1f} ms (cum {TIMINGS[name]*1000:.1f} ms)")

def report_timings():
    print("\n====== Timing Summary (ms) ======")
    for k, v in sorted(TIMINGS.items(), key=lambda kv: kv[1], reverse=True):
        print(f"{k:30s} {v*1000:10.1f}")
    try:
        df = pd.DataFrame([{"phase": k, "ms": v*1000} for k, v in TIMINGS.items()]).sort_values("ms", ascending=False)
        df.to_excel("timings.xlsx", index=False)
        print("[저장] timings.xlsx")
    except Exception as e:
        print(f"[요약 저장 실패] {e}")

atexit.register(report_timings)

# ======================= 위험 관련 =======================
DANGER_RADIUS_METERS = {
    'road_collapse': 10,
    'bridge_collapse': 10,
    'tanks': 300,
    'enemies': 200,
    'fire': 300,
    'explosion': 300,
    'barbed_wire': 10,
    'rockfall': 10
}
DANGER_COLORS = {
    'road_collapse': 'red',
    'bridge_collapse': 'darkred',
    'tanks': 'brown',
    'enemies': 'orange',
    'fire': 'purple',
    'explosion': 'black',
    'barbed_wire': 'green',
    'rockfall': 'pink'
}
RISK_WEIGHTS = {
    'road_collapse': 10000,
    'bridge_collapse': 100000000,
    'tanks': 1000,
    'enemies': 500,
    'fire': 200,
    'explosion': 1000,
    'barbed_wire': 100000000,
    'rockfall': 100000000
}
ALPHA = 1.0

selected_danger_type = None
active_danger_zones = []
danger_circle_artists = []
affected_edge_artists = []
current_robot_node = None
cch = None
ax = None
G = None

# ======================= 휴리스틱 =======================
def heuristic(u, v):
    u_coord = (G.nodes[u]['y'], G.nodes[u]['x'])
    v_coord = (G.nodes[v]['y'], G.nodes[v]['x'])
    return geodesic(u_coord, v_coord).meters

# ======================= 가중치 함수 =======================
def get_custom_weight(G, danger_zones):
    def weight_fn(u, v, d):
        base = d.get("length", 1.0)
        risk_score = 0.0
        ex = (G.nodes[u]['x'] + G.nodes[v]['x']) / 2
        ey = (G.nodes[u]['y'] + G.nodes[v]['y']) / 2
        for z in danger_zones:
            cx, cy = z['center_lon'], z['center_lat']
            dist = ox.distance.great_circle(cy, cx, ey, ex)
            if dist <= z['effective_radius_meters']:
                risk_score += RISK_WEIGHTS.get(z['type'], 0.0)
        gat_weight = float(d.get('gat_weight', 0.0))
        return base + ALPHA * risk_score + gat_weight
    return weight_fn

# ======================= A* (카운트 포함) =======================
def astar_with_counts(Gs, s, t, heuristic, weight_attr='weight'):
    start_ms = time.time()
    openq = []
    heapq.heappush(openq, (0.0, s))
    came = {s: None}
    g = {s: 0.0}
    expanded = 0
    closed = set()

    while openq:
        _, u = heapq.heappop(openq)
        if u in closed:
            continue
        closed.add(u)
        expanded += 1
        if u == t:
            path = []
            cur = t
            while cur is not None:
                path.append(cur)
                cur = came[cur]
            path.reverse()
            ms = (time.time() - start_ms) * 1000.0
            return path, expanded, ms

        for v, data in Gs[u].items():
            w = data.get(weight_attr, 1.0)
            cand = g[u] + w
            if cand < g.get(v, float('inf')):
                g[v] = cand
                f = cand + heuristic(v, t)
                heapq.heappush(openq, (f, v))
                came[v] = u

    ms = (time.time() - start_ms) * 1000.0
    raise nx.NetworkXNoPath(f"No path from {s} to {t} (expanded={expanded}, {ms:.1f}ms)")

# ======================= CCH 클래스 =======================
class CCH:
    def __init__(self, G):
        self.G = G
        self.order = []
        self.rank = {}
        self.shortcuts = nx.DiGraph()
        self.weight_fn = lambda u, v, d: d.get('length', 1.0)
        self._last_sig = None  # 위험존 변경 감지용

    def _danger_signature(self, zones):
        # 위험존 리스트 → 해시 가능한 서명(작게 반올림해서 과민 반응 방지)
        return tuple(sorted(
            (z['type'], round(z['center_lon'], 5), round(z['center_lat'], 5), int(z['effective_radius_meters']))
            for z in zones
        ))

    def build_hierarchy(self):
        self.order = sorted(self.G.nodes(), key=lambda n: len(self.G[n]))
        self.rank = {n: i for i, n in enumerate(self.order)}
        self.shortcuts.clear()

        # 기본 숏컷(두 엣지 합) 생성
        for v in self.order:
            nbrs = list(self.G[v])
            for i in range(len(nbrs)):
                for j in range(i + 1, len(nbrs)):
                    u, w = nbrs[i], nbrs[j]
                    if not (self.G.has_edge(u, v) and self.G.has_edge(v, w)):
                        continue
                    if self.rank[u] < self.rank[v] and self.rank[w] < self.rank[v]:
                        continue
                    c1 = self.weight_fn(u, v, self.G[u][v])
                    c2 = self.weight_fn(v, w, self.G[v][w])
                    tot = c1 + c2
                    if self.shortcuts.has_edge(u, w):
                        if tot < self.shortcuts[u][w]['weight']:
                            self.shortcuts[u][w]['weight'] = tot
                    else:
                        self.shortcuts.add_edge(u, w, weight=tot)

        # 원 엣지도 포함
        for u, w, data in self.G.edges(data=True):
            wgt = self.weight_fn(u, w, data)
            if self.shortcuts.has_edge(u, w):
                if wgt < self.shortcuts[u][w]['weight']:
                    self.shortcuts[u][w]['weight'] = wgt
            else:
                self.shortcuts.add_edge(u, w, weight=wgt)

    def customize(self, weight_fn, danger_zones, mode="fast"):
        self.weight_fn = weight_fn
        sig = self._danger_signature(danger_zones)

        # 위험이 없고, GAT도 변화 없으면 스킵 (초기 속도 ↑)
        if not danger_zones and sig == self._last_sig:
            return
        self._last_sig = sig

        if mode == "exact":
            # 기존 방식: 모든 숏컷 A* 재평가 (느림)
            for u, v in list(self.shortcuts.edges()):
                try:
                    cost = nx.astar_path_length(self.G, u, v, heuristic=heuristic, weight=weight_fn)
                except nx.NetworkXNoPath:
                    cost = float('inf')
                self.shortcuts[u][v]['weight'] = cost
        else:
            # FAST: 수축 당시와 같이 두 엣지 합으로 재구성 (매우 빠름)
            self.build_hierarchy()


    def query(self, s, t):
        path, expanded, ms = astar_with_counts(self.shortcuts, s, t, heuristic, 'weight')
        return path, expanded, ms

# ======================= 기타 보조 =======================
def convert_radius_meters_to_degrees(radius_meters, lat):
    lat_deg = radius_meters / 111139.0
    lon_deg = radius_meters / (111139.0 * math.cos(math.radians(lat)))
    return lat_deg, lon_deg

def edge_midpoint_xy(G, u, v):
    return ((G.nodes[u]['x'] + G.nodes[v]['x']) / 2.0,
            (G.nodes[u]['y'] + G.nodes[v]['y']) / 2.0)

def edge_is_in_any_hazard(G, u, v, danger_zones):
    ex, ey = edge_midpoint_xy(G, u, v)
    for z in danger_zones:
        cx, cy = z['center_lon'], z['center_lat']
        if ox.distance.great_circle(cy, cx, ey, ex) <= z['effective_radius_meters']:
            return True, z['type']
    return False, None

def edge_base_len_and_gat(G, u, v):
    data = G.get_edge_data(u, v, default={})
    if isinstance(data, dict) and 0 in data:
        data = data[0]
    base = float(data.get('length', 0.0))
    gat = float(data.get('gat_weight', 0.0))
    return base, gat

# ======================= GAT 적용 =======================
def add_gat_weights_to_graph(G, gat_df):
    print("GAT 위험도 가중치 추가 중...")
    risk_lookup = {
        (int(row['u']), int(row['v'])): float(row['risk_score_pred'])
        for _, row in gat_df.iterrows()
    }
    for u, v, data in G.edges(data=True):
        data['gat_weight'] = float(risk_lookup.get((u, v), 0.0))
    print("GAT 가중치 추가 완료.")

# ======================= 메인 =======================
if __name__ == '__main__':
    GRAPH_PATH = "seoul_gangnam_graph.pkl"

    # 그래프 로딩/생성
    with timer("graph_load_or_download"):
        if os.path.exists(GRAPH_PATH):
            with open(GRAPH_PATH, 'rb') as f:
                G = pickle.load(f)
        else:
            G = ox.graph_from_place("Gangnam-gu, Seoul, South Korea", network_type='all')
            with open(GRAPH_PATH, 'wb') as f:
                pickle.dump(G, f)
    print(f"노드 수: {len(G.nodes)} | 엣지 수: {len(G.edges)}")

    # GAT CSV
    try:
        with timer("GAT_csv_read"):
            gat_df = pd.read_csv(r"D:\workspaces\15_Final PJT\GAT\GATdanger.csv")
        with timer("GAT_apply_to_edges"):
            add_gat_weights_to_graph(G, gat_df)
    except Exception as e:
        print(f"[알림] GAT CSV 미사용({e}). 모든 'gat_weight'=0.0")
        for _, _, d in G.edges(data=True):
            d['gat_weight'] = 0.0

    # 출발/도착
    s_lat, s_lon = 37.526594, 127.028545
    g_lat, g_lon = 37.484410, 127.054338
    with timer("nearest_node_start"):
        start = ox.distance.nearest_nodes(G, s_lon, s_lat)
    with timer("nearest_node_goal"):
        goal = ox.distance.nearest_nodes(G, g_lon, g_lat)
    current_robot_node = start

    # CCH
    with timer("cch_build_hierarchy"):
        cch = CCH(G)
        cch.build_hierarchy()
    with timer("cch_customize_initial"):
        cch.customize(get_custom_weight(G, active_danger_zones),active_danger_zones, mode="fast")

    # 시각화
    with timer("plot_graph_initial"):
        fig, ax = ox.plot_graph(G, show=False, close=False, node_size=0,
                                edge_color='gray', edge_linewidth=0.5)
    with timer("add_basemap_initial"):
        cx.add_basemap(ax, crs=G.graph['crs'], source=cx.providers.OpenStreetMap.Mapnik, zoom=12)

    ax.scatter(G.nodes[start]['x'], G.nodes[start]['y'], c='lime', s=200, label='출발')
    ax.scatter(G.nodes[goal]['x'], G.nodes[goal]['y'], c='purple', s=200, label='도착')
    robot_line, = ax.plot([], [], c='blue', lw=4, label='로봇 이동')
    plan_line,  = ax.plot([], [], c='orange', lw=2, label='현재 계획')
    robot_marker = ax.scatter([], [], c='cyan', s=150, marker='s', label='현재 위치')
    ax.legend()

    plt.ion()
    plt.show()

    # 시뮬 루프
    robot_path = [start]
    cur = start
    step = 0
    max_steps = 50

    while cur != goal and step < max_steps:
        step += 1
        with timer("query_cch_astar"):
            path, expanded, ms = cch.query(cur, goal)
        next_node = path[1]
        cur = next_node
        robot_path.append(cur)

        with timer("draw_update_lines"):
            rx = [G.nodes[n]['x'] for n in robot_path]
            ry = [G.nodes[n]['y'] for n in robot_path]
            robot_line.set_data(rx, ry)
            px = [G.nodes[n]['x'] for n in path]
            py = [G.nodes[n]['y'] for n in path]
            plan_line.set_data(px, py)
            robot_marker.set_offsets((G.nodes[cur]['x'], G.nodes[cur]['y']))
            ax.set_title(f"STEP {step}: 현재 {cur}")
            plt.draw()
        with timer("ui_pause"):
            plt.pause(0.5)

    plt.ioff()
    plt.show()


[⏱] graph_load_or_download: 268.2 ms (cum 268.2 ms)
노드 수: 8307 | 엣지 수: 22547
[⏱] GAT_csv_read: 1.1 ms (cum 1.1 ms)
[알림] GAT CSV 미사용([Errno 2] No such file or directory: 'D:\\workspaces\\15_Final PJT\\GAT\\GATdanger.csv'). 모든 'gat_weight'=0.0
[⏱] nearest_node_start: 46.1 ms (cum 46.1 ms)
[⏱] nearest_node_goal: 45.0 ms (cum 45.0 ms)
[⏱] cch_build_hierarchy: 921.5 ms (cum 921.5 ms)
[⏱] cch_customize_initial: 713.9 ms (cum 713.9 ms)
[⏱] plot_graph_initial: 4722.5 ms (cum 4722.5 ms)
[⏱] add_basemap_initial: 159.4 ms (cum 159.4 ms)
[⏱] query_cch_astar: 158.4 ms (cum 158.4 ms)
[⏱] draw_update_lines: 0.7 ms (cum 0.7 ms)
[⏱] ui_pause: 667.4 ms (cum 667.4 ms)
[⏱] query_cch_astar: 129.3 ms (cum 287.7 ms)
[⏱] draw_update_lines: 0.6 ms (cum 1.4 ms)
[⏱] ui_pause: 502.5 ms (cum 1169.9 ms)
[⏱] query_cch_astar: 170.9 ms (cum 458.6 ms)
[⏱] draw_update_lines: 0.6 ms (cum 1.9 ms)
[⏱] ui_pause: 507.5 ms (cum 1677.3 ms)
[⏱] query_cch_astar: 186.4 ms (cum 645.0 ms)
[⏱] draw_update_lines: 0.7 ms (cum 2.6 ms)


In [ ]:
# 캐시 활성화
ox.settings.use_cache = True   # 다운로드한 OSM 데이터 캐싱
ox.settings.log_console = False

# 타일 캐시 디렉토리 지정 (프로젝트 안에 생성됨)
cx.set_cache_dir(os.path.join(os.getcwd(), "tile_cache"))

# ========= 위험 유형별 반경/색상/가중치 ==========
DANGER_RADIUS_METERS = {
    'road_collapse': 10,
    'bridge_collapse': 10,
    'tanks': 300,
    'enemies': 200,
    'fire': 300,
    'explosion': 300,
    'barbed_wire': 10,
    'rockfall': 10
}
DANGER_COLORS = {
    'road_collapse': 'red',
    'bridge_collapse': 'darkred',
    'tanks': 'brown',
    'enemies': 'orange',
    'fire': 'purple',
    'explosion': 'black',
    'barbed_wire': 'green',
    'rockfall': 'pink'
}
RISK_WEIGHTS = {
    'road_collapse': 10000,
    'bridge_collapse': 100000000,
    'tanks': 1000,
    'enemies': 500,
    'fire': 200,
    'explosion': 1000,
    'barbed_wire': 100000000,
    'rockfall': 100000000
}
ALPHA = 1.0  # 위험 반영 계수

# ========= 전역 상태 ==========
selected_danger_type = None
active_danger_zones = []
danger_circle_artists = []
affected_edge_artists = []
current_robot_node = None
cch = None
ax = None
G = None

# ========= 휴리스틱 (지오데식) ==========
def heuristic(u, v):
    u_coord = (G.nodes[u]['y'], G.nodes[u]['x'])
    v_coord = (G.nodes[v]['y'], G.nodes[v]['x'])
    return geodesic(u_coord, v_coord).meters

# ========= 엣지 가중치(위험+GAT) ==========
def get_custom_weight(G, danger_zones):
    def weight_fn(u, v, d):
        base = d.get("length", 1.0)
        risk_score = 0.0
        # 엣지 중점 기준 위험 반경 체크
        ex = (G.nodes[u]['x'] + G.nodes[v]['x']) / 2
        ey = (G.nodes[u]['y'] + G.nodes[v]['y']) / 2
        for z in danger_zones:
            cx, cy = z['center_lon'], z['center_lat']
            dist = ox.distance.great_circle(cy, cx, ey, ex)
            if dist <= z['effective_radius_meters']:
                risk_score += RISK_WEIGHTS.get(z['type'], 0.0)
        gat_weight = float(d.get('gat_weight', 0.0))
        return base + ALPHA * risk_score + gat_weight
    return weight_fn

# ========= 카운팅 A* (확장노드/시간) ==========
def astar_with_counts(Gs, s, t, heuristic, weight_attr='weight'):
    start_ms = time.time()
    openq = []
    heapq.heappush(openq, (0.0, s))
    came = {s: None}
    g =0 {s: 0.0}
    expanded = 0
    closed = set()

    while openq:
        _, u = heapq.heappop(openq)
        if u in closed:
            continue
        closed.add(u)
        expanded += 1
        if u == t:
            # 경로 복원
            path = []
            cur = t
            while cur is not None:
                path.append(cur)
                cur = came[cur]
            path.reverse()
            ms = (time.time() - start_ms) * 100.0
            return path, expanded, ms

        for v, data in Gs[u].items():
            w = data.get(weight_attr, 1.0)
            cand = g[u] + w
            if cand < g.get(v, float('inf')):
                g[v] = cand
                f = cand + heuristic(v, t)
                heapq.heappush(openq, (f, v))
                came[v] = u

    ms = (time.time() - start_ms) * 1000.0
    raise nx.NetworkXNoPath(f"No path from {s} to {t} (expanded={expanded}, {ms:.1f}ms)")

# ========= CCH (단순화 버전) ==========
class CCH:
    def __init__(self, G):
        self.G = G
        self.order = []
        self.rank = {}
        self.shortcuts = nx.DiGraph()
        self.weight_fn = lambda u, v, d: d.get('length', 1.0)
        self._last_sig = None  # 위험존 변경 감지용

    def _danger_signature(self, zones):
        return tuple(sorted(
            (z['type'], round(z['center_lon'], 5), round(z['center_lat'], 5), int(z['effective_radius_meters']))
            for z in zones
        ))

    def _edge_cost(self, u, v):
        """MultiDiGraph 안전하게 가중치 뽑기"""
        data = self.G.get_edge_data(u, v, default=None)
        if data is None:
            return float('inf')
        # MultiDiGraph일 때: key -> dict 구조
        if isinstance(data, dict) and all(isinstance(val, dict) for val in data.values()) and 'length' not in data:
            return min(self.weight_fn(u, v, attr) for attr in data.values())
        return self.weight_fn(u, v, data)

    def build_hierarchy(self):
        self.order = sorted(self.G.nodes(), key=lambda n: len(self.G[n]))
        self.rank = {n: i for i, n in enumerate(self.order)}
        self.shortcuts.clear()

        for v in self.order:
            nbrs = list(self.G[v])
            for i in range(len(nbrs)):
                for j in range(i + 1, len(nbrs)):
                    u, w = nbrs[i], nbrs[j]
                    if not (self.G.has_edge(u, v) and self.G.has_edge(v, w)):
                        continue
                    if self.rank[u] < self.rank[v] and self.rank[w] < self.rank[v]:
                        continue
                    c1 = self._edge_cost(u, v)
                    c2 = self._edge_cost(v, w)
                    tot = c1 + c2
                    if self.shortcuts.has_edge(u, w):
                        if tot < self.shortcuts[u][w]['weight']:
                            self.shortcuts[u][w]['weight'] = tot
                    else:
                        self.shortcuts.add_edge(u, w, weight=tot)

        # 원래 엣지도 포함 (Multi/DiGraph 둘 다)
        if isinstance(self.G, (nx.MultiGraph, nx.MultiDiGraph)):
            for u, w, key, data in self.G.edges(keys=True, data=True):
                wgt = self.weight_fn(u, w, data)
                if self.shortcuts.has_edge(u, w):
                    if wgt < self.shortcuts[u][w]['weight']:
                        self.shortcuts[u][w]['weight'] = wgt
                else:
                    self.shortcuts.add_edge(u, w, weight=wgt)
        else:
            for u, w, data in self.G.edges(data=True):
                wgt = self.weight_fn(u, w, data)
                if self.shortcuts.has_edge(u, w):
                    if wgt < self.shortcuts[u][w]['weight']:
                        self.shortcuts[u][w]['weight'] = wgt
                else:
                    self.shortcuts.add_edge(u, w, weight=wgt)

    def customize(self, weight_fn, danger_zones, mode="fast"):
        self.weight_fn = weight_fn
        sig = self._danger_signature(danger_zones)

        # 위험이 없고, GAT도 변화 없으면 스킵 (초기 속도 ↑)
        if not danger_zones and sig == self._last_sig:
            return
        self._last_sig = sig

        if mode == "exact":
            # 기존 방식: 모든 숏컷 A* 재평가 (느림)
            for u, v in list(self.shortcuts.edges()):
                try:
                    cost = nx.astar_path_length(self.G, u, v, heuristic=heuristic, weight=weight_fn)
                except nx.NetworkXNoPath:
                    cost = float('inf')
                self.shortcuts[u][v]['weight'] = cost
        else:
            # FAST: 수축 당시와 같이 두 엣지 합으로 재구성 (매우 빠름)
            self.build_hierarchy()


    def query(self, s, t):
        path, expanded, ms = astar_with_counts(self.shortcuts, s, t, heuristic, 'weight')
        return path, expanded, ms

# ========= 보조 함수 (반경/위경 변환, 위험검사, 길이/GAT) ==========
def convert_radius_meters_to_degrees(radius_meters, lat):
    lat_deg = radius_meters / 111139.0
    lon_deg = radius_meters / (111139.0 * math.cos(math.radians(lat)))
    return lat_deg, lon_deg

def edge_midpoint_xy(G, u, v):
    return ((G.nodes[u]['x'] + G.nodes[v]['x']) / 2.0,
            (G.nodes[u]['y'] + G.nodes[v]['y']) / 2.0)

def edge_is_in_any_hazard(G, u, v, danger_zones):
    ex, ey = edge_midpoint_xy(G, u, v)
    for z in danger_zones:
        cx, cy = z['center_lon'], z['center_lat']
        if ox.distance.great_circle(cy, cx, ey, ex) <= z['effective_radius_meters']:
            return True, z['type']
    return False, None

def edge_base_len_and_gat(G, u, v):
    data = G.get_edge_data(u, v, default={})
    if isinstance(data, dict) and 0 in data:  # MultiGraph 대응
        data = data[0]
    base = float(data.get('length', 0.0))
    gat = float(data.get('gat_weight', 0.0))
    return base, gat

# ========= 위험 유형 선택/그리기 ==========
def select_danger_type_prompt():
    global selected_danger_type
    print("\n위험 유형 선택:")
    print("1. road_collapse  2. bridge_collapse  3. tanks  4. enemies")
    print("5. fire           6. explosion        7. barbed_wire  8. rockfall")
    print("9. 선택 해제")
    choice = input("선택 (1~9): ")
    selected_danger_type = {
        '1': 'road_collapse', '2': 'bridge_collapse', '3': 'tanks', '4': 'enemies',
        '5': 'fire', '6': 'explosion', '7': 'barbed_wire', '8': 'rockfall', '9': None
    }.get(choice, None)
    print(f"선택된 위험: {selected_danger_type}")

def redraw_danger_zones(G, ax):
    global danger_circle_artists, affected_edge_artists
    for artist in danger_circle_artists + affected_edge_artists:
        try:
            artist.remove()
        except Exception:
            pass
    danger_circle_artists.clear()
    affected_edge_artists.clear()

    for z in active_danger_zones:
        cx, cy = z['center_lon'], z['center_lat']
        radius = z['effective_radius_meters']
        color = DANGER_COLORS.get(z['type'], 'gray')

        lat_off, lon_off = convert_radius_meters_to_degrees(radius, cy)
        circle_lons, circle_lats = [], []
        for i in range(100):
            ang = 2 * math.pi * i / 100
            circle_lons.append(cx + lon_off * math.cos(ang))
            circle_lats.append(cy + lat_off * math.sin(ang))
        h, = ax.plot(circle_lons, circle_lats, c=color, ls='--', lw=2, zorder=7)
        danger_circle_artists.append(h)

# ========= 마우스/키보드 이벤트 ==========
def on_click(event):
    global selected_danger_type, active_danger_zones, cch, ax
    if event.inaxes != ax:
        return
    if event.button == 1 and selected_danger_type:
        cx0, cy0 = event.xdata, event.ydata
        r = DANGER_RADIUS_METERS[selected_danger_type]
        active_danger_zones.append({
            'type': selected_danger_type,
            'center_lon': cx0,
            'center_lat': cy0,
            'effective_radius_meters': r
        })
        print(f"➡ 위험 추가: {selected_danger_type} @ ({cx0:.5f}, {cy0:.5f})")
        cch.customize(get_custom_weight(G, active_danger_zones),active_danger_zones, mode="fast")
        redraw_danger_zones(G, ax)
        plt.draw()
    elif event.button == 3:
        print("🧹 위험 초기화")
        active_danger_zones.clear()
        cch.customize(get_custom_weight(G, active_danger_zones),active_danger_zones, mode="fast")
        redraw_danger_zones(G, ax)
        plt.draw()

def on_key(event):
    if event.key == 't':
        select_danger_type_prompt()

# ========= GAT 가중치 주입 ==========
def add_gat_weights_to_graph(G, gat_df):
    print("GAT 위험도 가중치 추가 중...")
    risk_lookup = {
        (int(row['u']), int(row['v'])): float(row['risk_score_pred'])
        for _, row in gat_df.iterrows()
    }
    for u, v, data in G.edges(data=True):
        data['gat_weight'] = float(risk_lookup.get((u, v), 0.0))
    print("GAT 가중치 추가 완료.")

# ========= 메인 ==========
if __name__ == '__main__':
    GRAPH_PATH = "seoul_gangnam_graph.pkl"

    # ---- 그래프 로딩/생성 ----
    if os.path.exists(GRAPH_PATH):
        print("저장된 그래프 불러오는 중...")
        with open(GRAPH_PATH, 'rb') as f:
            G = pickle.load(f)
    else:
        print("강남구 그래프 다운로드 중...")
        G = ox.graph_from_place("Gangnam-gu, Seoul, South Korea", network_type='all')
        print("그래프 저장 중...")
        with open(GRAPH_PATH, 'wb') as f:
            pickle.dump(G, f)
    print(f"노드 수: {len(G.nodes)} | 엣지 수: {len(G.edges)}")

    # ---- GAT CSV (옵션) ----
    try:
        gat_df = pd.read_csv(r"E:\workspaces\15_Final_PJT\04_EDA\GAT\GATdanger.csv")
        add_gat_weights_to_graph(G, gat_df)
    except Exception as e:
        print(f"[알림] GAT CSV 미사용({e}). 모든 'gat_weight' = 0.0")
        for _, _, d in G.edges(data=True):
            d['gat_weight'] = 0.0

    # ---- 출발/도착 ----
    s_lat, s_lon = 37.526594, 127.028545 # 압구정역
    g_lat, g_lon = 37.484410, 127.054338  # 개포동 경남아파트
    start = ox.distance.nearest_nodes(G, s_lon, s_lat)
    goal  = ox.distance.nearest_nodes(G, g_lon, g_lat)
    current_robot_node = start

    # ---- CCH ----
    cch = CCH(G)
    cch.build_hierarchy()
    cch.customize(get_custom_weight(G, active_danger_zones), active_danger_zones, mode="fast")

    # ---- 시각화 ----
    fig, ax = ox.plot_graph(G, show=False, close=False, node_size=0, edge_color='gray', edge_linewidth=0.5)
    cx.add_basemap(ax, crs=G.graph['crs'], source=cx.providers.OpenStreetMap.Mapnik, zoom=12)
    ax.scatter(G.nodes[start]['x'], G.nodes[start]['y'], c='lime', s=200, label='출발')
    ax.scatter(G.nodes[goal]['x'],  G.nodes[goal]['y'],  c='purple', s=200, label='도착')
    robot_line, = ax.plot([], [], c='blue', lw=4, label='로봇 이동')
    plan_line,  = ax.plot([], [], c='orange', lw=2, label='현재 계획')
    robot_marker = ax.scatter([], [], c='cyan', s=150, marker='s', label='현재 위치')
    ax.legend()

    fig.canvas.mpl_connect('button_press_event', on_click)
    fig.canvas.mpl_connect('key_press_event', on_key)
    plt.ion()
    plt.show()

    # ======== 메트릭 누적 변수 ========
    total_search_ms = 0.0
    total_expanded  = 0
    total_path_len_m = 0.0
    hazard_edges = 0
    safe_edges   = 0
    cum_risk_cost = 0.0
    cum_gat_cost  = 0.0
    replan_count  = 0
    deviation_sum = 0.0
    plan_stability_kept = 0
    prev_plan = None

    # ---- 시뮬 루프 ----
    robot_path = [start]
    cur = start
    step = 0
    max_steps = 150

    while cur != goal and step < max_steps:
        step += 1
        print(f"\n[STEP {step}] 현재 위치: {cur}")

        # 경로 질의 (탐색 시간/확장 노드 수 획득)
        try:
            path, expanded, ms = cch.query(cur, goal)
        except nx.NetworkXNoPath:
            print("❌ [경로 없음] 위험 요소 때문에 목적지 도달 불가")
            while True:
                resp = input("계속 시도하시겠습니까? (y/n): ").lower()
                if resp == 'y':
                    print("⏸️ 대기 중... 위험 조정 후 재계산")
                    plt.pause(2)
                    break
                elif resp == 'n':
                    print("🛑 시뮬레이션 종료")
                    plt.ioff(); plt.close()
                    exit()
                else:
                    print("❗ y 또는 n 입력")
            continue

        total_search_ms += ms
        total_expanded  += expanded

        # 재계획 감지/편차
        if prev_plan is not None and len(prev_plan) > 1 and len(path) > 1:
            if prev_plan[1] == path[1]:
                plan_stability_kept += 1
            else:
                replan_count += 1
                K = min(10, len(prev_plan), len(path))
                diff = sum(1 for i in range(K) if prev_plan[i] != path[i])
                deviation_sum += diff / max(1, K)
        prev_plan = path

        if len(path) < 2:
            print("✅ 더 이상 이동할 노드 없음.")
            break

        next_node = path[1]

        # 이동 엣지 메트릭 누적
        in_hazard, hz_type = edge_is_in_any_hazard(G, cur, next_node, active_danger_zones)
        base_len, gat_w = edge_base_len_and_gat(G, cur, next_node)
        total_path_len_m += base_len
        cum_gat_cost += gat_w
        if in_hazard:
            hazard_edges += 1
            if hz_type is not None:
                cum_risk_cost += RISK_WEIGHTS.get(hz_type, 0.0)
        else:
            safe_edges += 1

        # 사용자 경고/진행 여부(위험 엣지일 때)
        if in_hazard:
            print(f"⚠️ 위험 엣지 탐지 ({cur} ➔ {next_node}) → {hz_type}")
            while True:
                ans = input("이 엣지를 통과하시겠습니까? (y/n): ").lower()
                if ans == 'y':
                    print("➡️ 통과합니다.")
                    break
                elif ans == 'n':
                    print("⏸️ 대기. 위험 제거 또는 경로 변경을 기다립니다.")
                    plt.pause(2)
                    step -= 1
                    next_node = None
                    break
                else:
                    print("❗ y 또는 n 입력만 허용")
            if next_node is None:
                continue

        # 실제 이동
        cur = next_node
        robot_path.append(cur)
        print(f"➡️ 이동: {robot_path[-2]} → {cur}")

        # 시각화 업데이트
        rx = [G.nodes[n]['x'] for n in robot_path]
        ry = [G.nodes[n]['y'] for n in robot_path]
        robot_line.set_data(rx, ry)
        px = [G.nodes[n]['x'] for n in path]
        py = [G.nodes[n]['y'] for n in path]
        plan_line.set_data(px, py)
        robot_marker.set_offsets((G.nodes[cur]['x'], G.nodes[cur]['y']))
        ax.set_title(f"STEP {step}: 현재 {cur}")
        redraw_danger_zones(G, ax)
        plt.draw()
        plt.pause(0.8)

        if cur == goal:
            print("🎉 목표 도달!")

    plt.ioff(); plt.show()

    # ======== 시뮬 종료: 지표 산출 & 엑셀 저장 ========
    reached = (cur == goal)
    success_pct = 100.0 if reached else 0.0
    total_edges = hazard_edges + safe_edges
    avoid_pct = (safe_edges / total_edges * 100.0) if total_edges > 0 else 0.0
    avg_dev = (deviation_sum / max(1, replan_count)) if replan_count > 0 else 0.0
    stability_pct = (plan_stability_kept / max(1, step) * 100.0) if step > 0 else 0.0

    row = {
        '맵 번호': 1,
        '출발지~목적지': f"{s_lat:.6f}, {s_lon:.6f} -> {g_lat:.6f}, {g_lon:.6f}",
        '알고리즘': 'CCH+A*',
        '총 경로 거리': round(total_path_len_m / 1000.0, 2),  # km
        '탐색 시간': round(total_search_ms, 1),               # ms (총합)
        '탐색 노드 수': int(total_expanded),
        '위험 회피율': round(avoid_pct, 1),                  # %
        '누적 위험 비용': round(cum_risk_cost + cum_gat_cost),
        '재계획 빈도': int(replan_count),
        '재계획 시 경로 편차': round(avg_dev, 3),            # 0~1
        '경로 안정성/연속성': round(stability_pct, 1),       # %
        '성공률': round(success_pct, 1),                      # %
        '위험 유형': ",".join(sorted({z['type'] for z in active_danger_zones})) if active_danger_zones else '',
        '위험 개수': len(active_danger_zones),
    }

    out_path = "sim_dataset.xlsx"
    try:
        if os.path.exists(out_path):
            old = pd.read_excel(out_path)
            new = pd.concat([old, pd.DataFrame([row])], ignore_index=True)
        else:
            new = pd.DataFrame([row])
        new.to_excel(out_path, index=False)
        print(f"[저장 완료] {out_path} ({len(new)}행)")
    except Exception as e:
        print(f"[저장 실패] {e}")


저장된 그래프 불러오는 중...
노드 수: 8307 | 엣지 수: 22547
GAT 위험도 가중치 추가 중...
GAT 가중치 추가 완료.

[STEP 1] 현재 위치: 8557584670
➡️ 이동: 8557584670 → 1780475838

[STEP 2] 현재 위치: 1780475838
➡️ 이동: 1780475838 → 4968626373

[STEP 3] 현재 위치: 4968626373
➡️ 이동: 4968626373 → 9891581228

[STEP 4] 현재 위치: 9891581228
➡️ 이동: 9891581228 → 9891581219

[STEP 5] 현재 위치: 9891581219
➡️ 이동: 9891581219 → 9891543509

[STEP 6] 현재 위치: 9891543509
➡️ 이동: 9891543509 → 9891581217

[STEP 7] 현재 위치: 9891581217
➡️ 이동: 9891581217 → 1853555169

[STEP 8] 현재 위치: 1853555169
➡️ 이동: 1853555169 → 4933675435

[STEP 9] 현재 위치: 4933675435
➡️ 이동: 4933675435 → 1780475794

[STEP 10] 현재 위치: 1780475794
➡️ 이동: 1780475794 → 5438813901

[STEP 11] 현재 위치: 5438813901
➡️ 이동: 5438813901 → 4179952622

[STEP 12] 현재 위치: 4179952622
➡️ 이동: 4179952622 → 7510250553

[STEP 13] 현재 위치: 7510250553
➡️ 이동: 7510250553 → 4179952620

[STEP 14] 현재 위치: 4179952620
➡️ 이동: 4179952620 → 4179952621

[STEP 15] 현재 위치: 4179952621
➡️ 이동: 4179952621 → 436861531

[STEP 16] 현재 위치: 436861531
➡️ 이